## Get statements about regulation of cell motility from INDRA DB

In [5]:
# INDRA can be installed with `pip install indra`
# GILDA can be installed with `pip install gilda`

# If you want to run this, let us know and we will set you up an API key

import csv
import json
from copy import deepcopy
from tqdm import tqdm
from indra.preassembler import Preassembler # For combining duplicate statements
from indra.sources import indra_db_rest as idr # For querying the INDRA DB REST API
from indra.tools import assemble_corpus as ac # Useful assembly and filtering functions
from indra.assemblers.html import HtmlAssembler # Rendering stmts as HTML
from indra.util import statement_presentation as sp # Rendering stmts as HTML
from indra.preassembler.grounding_mapper import gilda
from indra.statements import stmts_to_json, Activation, Inhibition


In [6]:
# List of motility target terms
motility_terms = ['GO:0048870@GO', # GO Cell Motility
                  'GO:0016477@GO'] # GO Cell migration (synonymous with MESH D002465)

In [7]:
# Get all statements for regulators of the various motility terms
last_proc = None
for motility_term in motility_terms:
    # Returns INDRA DB REST Processor object with all textual evidences
    idrp = idr.get_statements(object=motility_term, ev_limit=10)
    # Merge results information with results for previous term
    if last_proc is not None:
        idrp.merge_results(last_proc)
    last_proc = idrp

INFO: [2021-01-15 09:26:54] indra.sources.indra_db_rest.util - query: https://db.indra.bio/latest/statements/from_agents?object=GO:0048870@GO&offset=0
INFO: [2021-01-15 09:26:54] indra.sources.indra_db_rest.util - params: {'ev_limit': 10, 'best_first': True, 'api_key': '[api-key]'}
INFO: [2021-01-15 09:26:59] indra.sources.indra_db_rest.util - query: https://db.indra.bio/latest/statements/from_agents?object=GO:0048870@GO&offset=500
INFO: [2021-01-15 09:26:59] indra.sources.indra_db_rest.util - params: {'ev_limit': 10, 'best_first': True, 'api_key': '[api-key]'}
INFO: [2021-01-15 09:27:01] indra.sources.indra_db_rest.util - query: https://db.indra.bio/latest/statements/from_agents?object=GO:0048870@GO&offset=1000
INFO: [2021-01-15 09:27:01] indra.sources.indra_db_rest.util - params: {'ev_limit': 10, 'best_first': True, 'api_key': '[api-key]'}
INFO: [2021-01-15 09:27:03] indra.sources.indra_db_rest.util - query: https://db.indra.bio/latest/statements/from_agents?object=GO:0048870@GO&offs

INFO: [2021-01-15 09:28:01] indra.sources.indra_db_rest.util - query: https://db.indra.bio/latest/statements/from_agents?object=GO:0016477@GO&offset=9000
INFO: [2021-01-15 09:28:01] indra.sources.indra_db_rest.util - params: {'ev_limit': 10, 'best_first': True, 'api_key': '[api-key]'}
INFO: [2021-01-15 09:28:02] indra.sources.indra_db_rest.util - query: https://db.indra.bio/latest/statements/from_agents?object=GO:0016477@GO&offset=9500
INFO: [2021-01-15 09:28:02] indra.sources.indra_db_rest.util - params: {'ev_limit': 10, 'best_first': True, 'api_key': '[api-key]'}
INFO: [2021-01-15 09:28:04] indra.sources.indra_db_rest.util - query: https://db.indra.bio/latest/statements/from_agents?object=GO:0016477@GO&offset=10000
INFO: [2021-01-15 09:28:04] indra.sources.indra_db_rest.util - params: {'ev_limit': 10, 'best_first': True, 'api_key': '[api-key]'}
INFO: [2021-01-15 09:28:05] indra.sources.indra_db_rest.util - query: https://db.indra.bio/latest/statements/from_agents?object=GO:0016477@GO

Let's see how many statements we got

In [8]:
len(idrp.statements)

23754

Because we'll need it later, we build a mapping from the statement UUID to the source and evidence counts. The UUID doesn't change when we re-ground the statements, whereas the statement hash does, making it possible for us to retrieve the evidence counts when generating the tabular and HTML results.

In [9]:
ev_counts_by_hash = idrp.get_ev_counts()
ev_counts_by_uuid = {s.uuid: ev_counts_by_hash[s.get_hash()] for s in idrp.statements}
src_counts_by_hash = idrp.get_source_counts()
src_counts_by_uuid = {s.uuid: src_counts_by_hash[s.get_hash()] for s in idrp.statements}

Now make a copy of the statements and fix (in place, in the copy) a few cases of statements missing standard identifiers by grounding to IDs with Gilda (see http://grounding.indra.bio)

In [10]:
grounded_stmts = deepcopy(idrp.statements)

Now use GILDA to find standard identifiers for a subset of statements that the source reading systems couldn't normalize.

In [11]:
grounded_stmts = gilda.ground_statements(grounded_stmts, mode='local', ungrounded_only=True)

INFO: [2021-01-15 09:32:14] gilda.grounder - Looking up the following strings: sirna
INFO: [2021-01-15 09:32:14] gilda.grounder - Comparing siRNA with 1 entries
INFO: [2021-01-15 09:32:14] gilda.grounder - Running disambiguation for siRNA
INFO: [2021-01-15 09:32:14] indra.ontology.bio.ontology - Loading INDRA bio ontology from cache at /Users/johnbachman/.indra/bio_ontology/1.7/bio_ontology.pkl
INFO: [2021-01-15 09:32:23] gilda.grounder - Looking up the following strings: sdf1alpha, sdf1a, sdf 1alpha, sdf1α
INFO: [2021-01-15 09:32:23] gilda.grounder - Comparing SDF-1alpha with 2 entries
INFO: [2021-01-15 09:32:23] gilda.grounder - Running disambiguation for SDF-1alpha
INFO: [2021-01-15 09:32:23] gilda.grounder - Looking up the following strings: laminin
INFO: [2021-01-15 09:32:23] gilda.grounder - Comparing laminin with 1 entries
INFO: [2021-01-15 09:32:23] gilda.grounder - Running disambiguation for laminin
INFO: [2021-01-15 09:32:23] gilda.grounder - Looking up the following strings:

INFO: [2021-01-15 09:32:23] gilda.grounder - Looking up the following strings: ptpμ, ptpmu
INFO: [2021-01-15 09:32:23] gilda.grounder - Comparing PTPmu with 0 entries
INFO: [2021-01-15 09:32:23] gilda.grounder - Looking up the following strings: rantes
INFO: [2021-01-15 09:32:23] gilda.grounder - Comparing RANTES with 1 entries
INFO: [2021-01-15 09:32:23] gilda.grounder - Running disambiguation for RANTES
INFO: [2021-01-15 09:32:23] gilda.grounder - Looking up the following strings: uca1
INFO: [2021-01-15 09:32:23] gilda.grounder - Comparing UCA1 with 1 entries
INFO: [2021-01-15 09:32:23] gilda.grounder - Running disambiguation for UCA1
INFO: [2021-01-15 09:32:23] gilda.grounder - Looking up the following strings: mir 125b, mir125b
INFO: [2021-01-15 09:32:23] gilda.grounder - Comparing miR-125b with 0 entries
INFO: [2021-01-15 09:32:23] gilda.grounder - Looking up the following strings: mir143, mir 143
INFO: [2021-01-15 09:32:23] gilda.grounder - Comparing miR-143 with 1 entries
INFO: 

INFO: [2021-01-15 09:32:23] gilda.grounder - Looking up the following strings: mir223, mir 223
INFO: [2021-01-15 09:32:23] gilda.grounder - Comparing miR-223 with 1 entries
INFO: [2021-01-15 09:32:23] gilda.grounder - Running disambiguation for miR-223
INFO: [2021-01-15 09:32:23] gilda.grounder - Looking up the following strings: mir 200b, mir200b
INFO: [2021-01-15 09:32:23] gilda.grounder - Comparing miR-200b with 1 entries
INFO: [2021-01-15 09:32:23] gilda.grounder - Running disambiguation for miR-200b
INFO: [2021-01-15 09:32:23] gilda.grounder - Looking up the following strings: emt
INFO: [2021-01-15 09:32:23] gilda.grounder - Comparing EMT with 3 entries
INFO: [2021-01-15 09:32:23] gilda.grounder - Running disambiguation for EMT
INFO: [2021-01-15 09:32:23] gilda.grounder - Result from Adeft: {'HGNC:6179': 3.190383253599192e-06, 'MESH:D058750': 0.9998914999797809, 'ungrounded': 0.00010530963696547337}
INFO: [2021-01-15 09:32:23] gilda.grounder - Multiplying the score of "epithelial 

INFO: [2021-01-15 09:32:23] gilda.grounder - Comparing miR-183 with 1 entries
INFO: [2021-01-15 09:32:23] gilda.grounder - Running disambiguation for miR-183
INFO: [2021-01-15 09:32:23] gilda.grounder - Looking up the following strings: nsc23766
INFO: [2021-01-15 09:32:23] gilda.grounder - Comparing NSC23766 with 2 entries
INFO: [2021-01-15 09:32:23] gilda.grounder - Running disambiguation for NSC23766
INFO: [2021-01-15 09:32:23] gilda.grounder - Looking up the following strings: cbp501
INFO: [2021-01-15 09:32:23] gilda.grounder - Comparing CBP501 with 0 entries
INFO: [2021-01-15 09:32:23] gilda.grounder - Looking up the following strings: mir243p, mir 24 3p
INFO: [2021-01-15 09:32:23] gilda.grounder - Comparing miR-24-3p with 0 entries
INFO: [2021-01-15 09:32:23] gilda.grounder - Looking up the following strings: mir23b, mir 23b
INFO: [2021-01-15 09:32:23] gilda.grounder - Comparing miR-23b with 1 entries
INFO: [2021-01-15 09:32:23] gilda.grounder - Running disambiguation for miR-23b


INFO: [2021-01-15 09:32:23] gilda.grounder - Multiplying the score of "cytidine 5'-monophosphate" with 0.978
INFO: [2021-01-15 09:32:23] gilda.grounder - Multiplying the score of "phosphatidylcholine(1+)" with 0.978
INFO: [2021-01-15 09:32:23] gilda.grounder - Multiplying the score of "polycarbonate macromolecule" with 0.978
INFO: [2021-01-15 09:32:23] gilda.grounder - Multiplying the score of "phosphocholine group" with 0.978
INFO: [2021-01-15 09:32:23] gilda.grounder - Multiplying the score of "polycarbonate polymer" with 0.978
INFO: [2021-01-15 09:32:23] gilda.grounder - Multiplying the score of "PC" with 0.001
INFO: [2021-01-15 09:32:23] gilda.grounder - Multiplying the score of "PODXL" with 0.978
INFO: [2021-01-15 09:32:23] gilda.grounder - Looking up the following strings: laminin
INFO: [2021-01-15 09:32:23] gilda.grounder - Comparing Laminin with 1 entries
INFO: [2021-01-15 09:32:23] gilda.grounder - Running disambiguation for Laminin
INFO: [2021-01-15 09:32:23] gilda.grounder -

...

INFO: [2021-01-15 09:32:47] gilda.grounder - Comparing TRAM-34 with 1 entries
INFO: [2021-01-15 09:32:47] gilda.grounder - Running disambiguation for TRAM-34
INFO: [2021-01-15 09:32:47] gilda.grounder - Looking up the following strings: cd133
INFO: [2021-01-15 09:32:47] gilda.grounder - Comparing CD133 with 1 entries
INFO: [2021-01-15 09:32:47] gilda.grounder - Running disambiguation for CD133
INFO: [2021-01-15 09:32:47] gilda.grounder - Looking up the following strings: na,k atpase beta 1 subunit, na,katpase b 1 subunit, na,katpase β 1 subunit, na,katpase beta 1 subunit
INFO: [2021-01-15 09:32:47] gilda.grounder - Comparing Na,K-ATPase beta 1 -subunit with 0 entries
INFO: [2021-01-15 09:32:47] gilda.grounder - Looking up the following strings: nobiletin
INFO: [2021-01-15 09:32:47] gilda.grounder - Comparing nobiletin with 1 entries
INFO: [2021-01-15 09:32:47] gilda.grounder - Running disambiguation for nobiletin
INFO: [2021-01-15 09:32:47] gilda.grounder - Looking up the following str

INFO: [2021-01-15 09:32:47] gilda.grounder - Looking up the following strings: slides were coated with polyllysine to increase adhesion, slides were coated with poly l lysine to increase adhesion
INFO: [2021-01-15 09:32:47] gilda.grounder - Comparing Slides were coated with poly-L-lysine to increase adhesion with 0 entries
INFO: [2021-01-15 09:32:47] gilda.grounder - Looking up the following strings: rhoacckvl, rhoa cckvl, ρacckvl
INFO: [2021-01-15 09:32:47] gilda.grounder - Comparing RhoA-CCKVL with 0 entries
INFO: [2021-01-15 09:32:47] gilda.grounder - Looking up the following strings: hotair knockdown
INFO: [2021-01-15 09:32:47] gilda.grounder - Comparing HOTAIR knockdown with 0 entries
INFO: [2021-01-15 09:32:47] gilda.grounder - Looking up the following strings: alphaadducinaa, aadducinaa, αadducinaa, alpha adducin aa
INFO: [2021-01-15 09:32:47] gilda.grounder - Comparing alpha-adducin-AA with 0 entries
INFO: [2021-01-15 09:32:47] gilda.grounder - Looking up the following strings:

INFO: [2021-01-15 09:32:47] gilda.grounder - Looking up the following strings: linc ubc1, lincubc1
INFO: [2021-01-15 09:32:47] gilda.grounder - Comparing linc-UBC1 with 1 entries
INFO: [2021-01-15 09:32:47] gilda.grounder - Running disambiguation for linc-UBC1
INFO: [2021-01-15 09:32:47] gilda.grounder - Looking up the following strings: na,kbeta, na,k beta, na,kβ, na,kb
INFO: [2021-01-15 09:32:47] gilda.grounder - Comparing Na,K-Beta with 0 entries
INFO: [2021-01-15 09:32:47] gilda.grounder - Looking up the following strings: insulinlike growth factor (igf1, insulin like growth factor (igf 1
INFO: [2021-01-15 09:32:47] gilda.grounder - Comparing insulin-like growth factor (IGF-1 with 0 entries
INFO: [2021-01-15 09:32:47] gilda.grounder - Looking up the following strings: βtrcp knockdown, beta trcp knock down, betatrcp knockdown, btrcp knockdown
INFO: [2021-01-15 09:32:47] gilda.grounder - Comparing beta-TrCP knock-down with 0 entries
INFO: [2021-01-15 09:32:47] gilda.grounder - Lookin

INFO: [2021-01-15 09:32:47] gilda.grounder - Running disambiguation for Ptx
INFO: [2021-01-15 09:32:47] gilda.grounder - Looking up the following strings: .3
INFO: [2021-01-15 09:32:47] gilda.grounder - Comparing .3 with 0 entries
INFO: [2021-01-15 09:32:47] gilda.grounder - Looking up the following strings: trksea receptor, trk sea receptor
INFO: [2021-01-15 09:32:47] gilda.grounder - Comparing Trk-Sea receptor with 0 entries
INFO: [2021-01-15 09:32:47] gilda.grounder - Looking up the following strings: fakρa, fakrhoa, fak rhoa
INFO: [2021-01-15 09:32:47] gilda.grounder - Comparing FAK-RhoA with 0 entries
INFO: [2021-01-15 09:32:47] gilda.grounder - Looking up the following strings: qss3
INFO: [2021-01-15 09:32:47] gilda.grounder - Comparing QSS3 with 0 entries
INFO: [2021-01-15 09:32:47] gilda.grounder - Looking up the following strings: cdc42h, cdc42hs
INFO: [2021-01-15 09:32:47] gilda.grounder - Comparing Cdc42Hs with 1 entries
INFO: [2021-01-15 09:32:47] gilda.grounder - Running d

INFO: [2021-01-15 09:32:47] gilda.grounder - Comparing extracellular-regulated kinases with 0 entries
INFO: [2021-01-15 09:32:47] gilda.grounder - Looking up the following strings: exosomal microrna mir1246, exosomal microrna mir 1246
INFO: [2021-01-15 09:32:47] gilda.grounder - Comparing Exosomal microRNA miR-1246 with 0 entries
INFO: [2021-01-15 09:32:47] gilda.grounder - Looking up the following strings: arm
INFO: [2021-01-15 09:32:47] gilda.grounder - Comparing arm with 2 entries
INFO: [2021-01-15 09:32:47] gilda.grounder - Running disambiguation for arm
INFO: [2021-01-15 09:32:47] gilda.grounder - Looking up the following strings: ct04
INFO: [2021-01-15 09:32:47] gilda.grounder - Comparing CT04 with 0 entries
INFO: [2021-01-15 09:32:47] gilda.grounder - Looking up the following strings: clpxp
INFO: [2021-01-15 09:32:47] gilda.grounder - Comparing ClpXP with 0 entries
INFO: [2021-01-15 09:32:47] gilda.grounder - Looking up the following strings: ehsp90α, ehsp90alpha, ehsp90a
INFO: 

## Filter to statements involving kinases

Load the kinase list from the TSV:

In [12]:
# List of kinases from Uniprot
with open('kinases.tsv', 'r') as f:
    csvreader = csv.reader(f, delimiter='\t')
    next(csvreader) # Skip the header row
    kinases = [gene_name for up_id, gene_name, species, up_name
                         in csvreader]

Let's take a look at processed the kinase list--could use other kinase lists as well.

In [13]:
kinases[:10]

['AAK1',
 'PRKAA1',
 'PRKAA2',
 'ABL1',
 'ABL2',
 'ACAD10',
 'ACAD11',
 'TNK2',
 'ACVR1B',
 'ACVR1C']

How many kinases?

In [14]:
len(kinases)

539

Filter to statements containing at least one kinase (in this case the subject), and allow families that have the genes as members, e.g. the families "ERK", "AKT", etc. (these are denoted by FamPlex identifiers)

In [15]:
kin_stmts = ac.filter_gene_list(grounded_stmts, kinases, 'one', allow_families=True, remove_bound=True)

INFO: [2021-01-15 09:33:52] indra.tools.assemble_corpus - Filtering 23754 statements for ones containing "one" of: AAK1, PRKAA1, PRKAA2, ABL1, ABL2, ACAD10, ACAD11, TNK2, ACVR1B, ACVR1C, ACVRL1, ACVR1, ADCK1, ADCK2, ADCK5, AKT1, AKT2, AKT3, ALK, ALPK1, ALPK2, ALPK3, AMHR2, ANKK1, NPR1, NPR2, ARAF, GRK2, GRK3, ATM, ATR, AURKA, AURKB, AURKC, ACVR2A, ACVR2B, BLK, BMP2K, BMPR2, BMPR1A, BMPR1B, BMX, BRAF, BRSK1, BRSK2, BTK, BUB1, BUB1B, CAMKV, CDK11A, CDK11B, CDC7, CDK10, CDK12, CDK13, CDK14, CDK15, CDK16, CDK17, CDK18, CDK19, CDK1, CDK20, CDK2, CDK3, CDK4, CDK5, CDK6, CDK7, CDK8, CDK9, CDKL1, CDKL2, CDKL3, CDKL4, CDKL5, CHEK1, CHEK2, CHKA, CHKB, CLK1, CLK2, CLK3, CLK4, COQ8A, COQ8B, CSF1R, CSNK2A1, CSNK2A2, CSNK2A3, CSK, CASK, CIT, DAPK1, DAPK2, DAPK3, DCLK1, DCLK2, DCLK3, DDR1, DDR2, DIPK1C, DMPK, DSTYK, DYRK1A, DYRK1B, DYRK2, DYRK3, DYRK4, EIF2AK1, EIF2AK2, EIF2AK3, EIF2AK4, EEF2K, EGFR, ETNK1, ETNK2, EPHA1, EPHA2, EPHA3, EPHA4, EPHA5, EPHA6, EPHA7, EPHA8, EPHA10, EPHB1, EPHB2, EPHB3, EP

How many statements did we get?

In [16]:
len(kin_stmts)

1042

## Assembling statements and summing evidence and source counts

Before we assemble, we should take note of statements that now have the same hash (i.e., same top-level structure) but different UUIDs (i.e., were originally distinct statements) due to the re-grounding that we did. We build a mapping dict so that we can subsequently add the evidence and source counts together for the statements that now overlap:

In [17]:
hash_to_uuids = {}
for stmt in kin_stmts:
    if stmt.get_hash(refresh=True) not in hash_to_uuids:
        hash_to_uuids[stmt.get_hash(refresh=True)] = []
    hash_to_uuids[stmt.get_hash(refresh=True)].append(stmt.uuid)

Now we use the INDRA Preassembler to combine evidences from statements that are now duplicates.

In [18]:
pa = Preassembler(None)
kin_stmts = pa.combine_duplicate_stmts(kin_stmts)
len(kin_stmts)

911

Because we will want to rank kinases by their overall evidence counts, we get the total evidence counts for each unique statement (determined by unique hash) by summing evidence counts from the originally distinct statements (stored by UUID):

In [19]:
# Now add up the evidence and source counts after assembly
final_ev_counts = {}
final_src_counts = {}

# Add up the evidences across the UUIDs for the statement
for stmt in kin_stmts:
    new_hash = stmt.get_hash(refresh=True)
    # Initialize the dict for this statement hash
    if new_hash not in final_ev_counts:
        final_ev_counts[new_hash] = 0
    if new_hash not in final_src_counts:
        final_src_counts[new_hash] = {}
    # Get all the previously distinct statements by UUID and add up
    # their evidence and source counts
    for uuid in hash_to_uuids[new_hash]:
        final_ev_counts[new_hash] += ev_counts_by_uuid[uuid] 
        for src_name, src_ct in src_counts_by_uuid[uuid].items():
            # Initialize the count for this source to 0
            if src_name not in final_src_counts[new_hash]:
                final_src_counts[new_hash][src_name] = 0
            # Add up all evidences for this source
            final_src_counts[new_hash][src_name] += src_ct

## Group by identifier type (gene or family) and save

Sort the statements into those that have a human gene as subject, a human protein family as subject, or neither (which we exclude).

In [20]:
hgnc_stmts = [] # HGNC kinase statements
fplx_stmts = [] # FPLX kinase statements
not_hgnc_fplx_stmts = [] # Statements with neither HGNC nor FPLX (only 2 of these)

for stmt in kin_stmts:
    # Get the identifiers for the subject/kinase agent
    ag_db_refs = stmt.agent_list()[0].db_refs
    # Check if the subject agent has an HGNC identifier
    if 'HGNC' in ag_db_refs:
        hgnc_stmts.append(stmt)
    # Check if it has a FPLX identifier (indicating a protein family)
    elif 'FPLX' in ag_db_refs:
        fplx_stmts.append(stmt)
    # Statement has neither HGNC or FPLX IDs
    else:
        not_hgnc_fplx_stmts = []

Save the statements as JSON:

In [21]:
with open('kinases_motility_hgnc.json', 'w') as f:
    json.dump(stmts_to_json(hgnc_stmts), f)
    
with open('kinases_motility_fplx.json', 'w') as f:
    json.dump(stmts_to_json(fplx_stmts), f)

## How many kinases do we find?

How many kinases do we find to be linked to motility?

In [22]:
hgnc_kin = set([stmt.agent_list()[0].name for stmt in kin_stmts
                if 'HGNC' in stmt.agent_list()[0].db_refs])
fplx_kin = set([stmt.agent_list()[0].name for stmt in kin_stmts
                if 'FPLX' in stmt.agent_list()[0].db_refs])

How many distinct genes linked to motility?

In [23]:
len(hgnc_kin)

313

How many protein families?

In [24]:
len(fplx_kin)

44

Names of the families:

In [25]:
fplx_kin

{'ABL_family',
 'AKT',
 'AMPK',
 'AMPK_alpha',
 'CAMK',
 'CAMK2_complex',
 'CDK',
 'CK2',
 'CSNK1',
 'DAPK',
 'DDR',
 'EPHB',
 'ERBB',
 'ERK',
 'Ephrin_receptor',
 'FGFR',
 'GSK3',
 'IKK_complex',
 'JAK',
 'JNK',
 'MAPK',
 'MEK',
 'MKNK',
 'Myosin_complex',
 'PAK',
 'PDGFR',
 'PI3K',
 'PI3K_p110',
 'PIM',
 'PKA',
 'PKC',
 'PRKG',
 'RAF',
 'ROCK',
 'ROR',
 'RSK',
 'RSTK1',
 'SRC',
 'TAT_associated_kinase',
 'TGFBR',
 'VEGFR',
 'mTORC1',
 'mTORC2',
 'p38'}

## Make a tabular dump of summary information

For handy reference, we make a tabular view of the HGNC and FPLX statements indicating

1. the kinase
2. Identifier namespace for the kinase (HGNC or FPLX)
3. Identifier (HGNC ID or FPLX ID, which is same as the FPLX name)
4. the number of total sentences for both "cell motility" and "cell migration"
5. the number of sentences for positive regulation of motility/migration
6. the number of sentences for negative regulation of motility/migration
7. a link out to results on INDRA DB for regulation of cell motility
8. a link out to results on INDRA DB for regulation of cell migration


In [26]:
def get_ev_counts_by_kinase(stmts, db_ns):
    counts = {}
    for stmt in stmts:
        kin_name = stmt.agent_list()[0].name
        db_id = stmt.agent_list()[0].db_refs[db_ns]
        stmt_ev_ct = final_ev_counts[stmt.get_hash(refresh=True)]
        motility_link = 'https://db.indra.bio/statements/from_agents?' \
                        f'subject={db_id}@{db_ns}&object=GO:0048870@GO&format=html'
        migration_link = 'https://db.indra.bio/statements/from_agents?' \
                        f'subject={db_id}@{db_ns}&object=GO:0016477@GO&format=html'
        if kin_name not in counts:
            counts[kin_name] = {'total_evidence': 0, 'pos_reg_evidence': 0,
                                'neg_reg_evidence': 0, 'db_ns': db_ns,
                                'db_id': db_id,
                                'motility_link': motility_link,
                                'migration_link': migration_link}
        counts[kin_name]['total_evidence'] += stmt_ev_ct
        if isinstance(stmt, Activation):
            counts[kin_name]['pos_reg_evidence'] += stmt_ev_ct
        elif isinstance(stmt, Inhibition):
            counts[kin_name]['neg_reg_evidence'] += stmt_ev_ct
    return counts

hgnc_counts = get_ev_counts_by_kinase(hgnc_stmts, 'HGNC')
fplx_counts = get_ev_counts_by_kinase(fplx_stmts, 'FPLX')

Write the results summaries to CSV files:

In [27]:
# Write the summary results to CSV files
def dump_summary_csv(counts_dict, filename):
    # CSV Header
    header = [['NAME', 'DB_NAMESPACE', 'DB_ID', 'TOTAL_EVIDENCE',
             'POS_REG_EVIDENCE', 'NEG_REG_EVIDENCE',
             'MOTILITY_LINK', 'MIGRATION_LINK']]
    data = [(kin_name, kin_dict['db_ns'], kin_dict['db_id'],
              kin_dict['total_evidence'],
             kin_dict['pos_reg_evidence'], kin_dict['neg_reg_evidence'],
             kin_dict['motility_link'], kin_dict['migration_link'])
            for kin_name, kin_dict in counts_dict.items()]
    # Sort by total evidence count
    data.sort(key=lambda x: x[3], reverse=True)
    with open(filename, 'w') as f:
        csvwriter = csv.writer(f, delimiter=',')
        csvwriter.writerows(header + data)

dump_summary_csv(hgnc_counts, 'kinase_motility_hgnc_summary.csv')
dump_summary_csv(fplx_counts, 'kinase_motility_fplx_summary.csv')

## Make HTML view of statements

To browse the text evidence linking each kinase to cell motility/migration, we generate an HTML view. We want to sort the results so the kinases with greater evidence of being link to either migration or motility show up at the top. To do this, we first total the evidences across all statements involving the same kinase.

In [31]:
# For sorting purposes, we add up all the evidences for the subject kinase
# across both GO terms
ev_by_subj = {}
for stmt in kin_stmts:
    stmt_subj = stmt.agent_list()[0].name    
    if stmt_subj not in ev_by_subj:
        ev_by_subj[stmt_subj] = 0
    ev_by_subj[stmt_subj] += final_ev_counts[stmt.get_hash()]

# We then apply these evidence totals to each individual statement
subj_ev_by_hash = {}
for stmt in kin_stmts:
    stmt_subj = stmt.agent_list()[0].name
    subj_ev = ev_by_subj[stmt_subj]
    subj_ev_by_hash[stmt.get_hash()] = subj_ev

Because this is an unusual type of sorting (grouping evidence counts by subject rather than by statement) we create a custom StmtStat object and sorting function to pass to the HTML Assembler:

In [32]:
# This aggregator passes the counts at the statement level to the statement
# group level without doing anything (no adding, averaging, etc.)
class EmptyAggregator(sp.BasicAggregator):
    def _merge(self, metric_array):
        self._values = metric_array

subj_ev_stat = sp.StmtStat('subject_ev', subj_ev_by_hash, int, EmptyAggregator)

# We sort by the subject evidence count first, and secondarily by the
# statement-level evidence count.
def sort_func(metrics):
    return (metrics['subject_ev'], metrics['ev_count'])

Initialize the HtmlAssembler object that will generate the HTML, using our assembled evidence and source counts, and custom subject-level evidence total, and sort function: 

In [33]:
ha = HtmlAssembler(kin_stmts, ev_counts=final_ev_counts, source_counts=final_src_counts,
                   title='Kinases regulating cell motility and migration', db_rest_url='https://db.indra.bio',
                   custom_stats=[subj_ev_stat], sort_by=sort_func)

In [34]:
ha.make_model(grouping_level='agent-pair')
ha.save_model('kinases_motility.html')

INFO: [2021-01-15 09:45:00] indra.assemblers.html.assembler - Removing MESH from refs due to too many matches: {'C496050', 'D051417'}
INFO: [2021-01-15 09:45:00] indra.assemblers.html.assembler - Removing MESH from refs due to too many matches: {'D020928', 'D048049'}
INFO: [2021-01-15 09:45:00] indra.assemblers.html.assembler - Removing NCIT from refs due to too many matches: {'C26360', 'C16376'}
INFO: [2021-01-15 09:45:00] indra.assemblers.html.assembler - Removing NCIT from refs due to too many matches: {'C26360', 'C16376'}
INFO: [2021-01-15 09:45:00] indra.assemblers.html.assembler - Removing MESH from refs due to too many matches: {'D019859', 'C491743'}
INFO: [2021-01-15 09:45:00] indra.assemblers.html.assembler - Removing MESH from refs due to too many matches: {'D019009', 'C000628936'}
INFO: [2021-01-15 09:45:00] indra.assemblers.html.assembler - Removing UP from refs due to too many matches: {'O00238', 'Q05438'}
